In [103]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [104]:
def to_hours(time):
    return (time // 3600) + 6

def clean_sales_data():
    df = pd.read_csv("../assets/example_data/sales_data.csv")
    sales_df = df[['token_id', 'sold_at', 'price']]
    sales_df = sales_df.rename(columns={'token_id': 'horse_id', 'sold_at': 'time_sold'}).sort_values('horse_id').reset_index(drop=True)
    sales_df['time_sold'] = sales_df['time_sold'].apply(to_hours)

    return sales_df

In [105]:
sales_data = clean_sales_data()

In [106]:
# sales_data['price'].hist()

In [107]:
eth_prices = pd.read_csv('../assets/eth_price_history.csv')
eth_prices.tail()

,Unnamed: 0,timestamp,eth_price
9091,9091,464923,1533.01
9092,9092,464924,1529.65
9093,9093,464925,1535.38
9094,9094,464926,1538.76
9095,9095,464927,1550.97


In [108]:
merged = pd.merge(sales_data, eth_prices, left_on='time_sold', right_on='timestamp', how='left')
merged['converted_price'] = merged['price'] * merged['eth_price']
clean_market_data = merged.drop(columns=['Unnamed: 0', 'price', 'timestamp','eth_price'])
clean_market_data

,horse_id,time_sold,converted_price
0,27,458373,60665.000
1,68,460370,13934.205
2,75,460370,9693.360
3,89,460370,9693.360
4,92,461981,10943.775
...,...,...,...
74906,520344,464862,139.492
74907,520345,464871,138.079
74908,520346,464863,139.884
74909,520404,464901,499.436


In [109]:
test_sales = clean_market_data[clean_market_data['horse_id'] == 154936]
test_sales

,horse_id,time_sold,converted_price
13115,154936,464926,15.3876
13116,154936,463789,11.6966


In [110]:
# horse data cleaning
horse_data = pd.read_csv('../assets/example_data/horse_meta_data.csv')
horse_data.columns
horse_data = horse_data.rename(columns={'data_horse_bloodline':'bloodline', 'data_horse_breed_type':'breed_type',
                           'data_horse_breeding_counter':'breeding_counter', 'data_horse_color':'horse_color',
                           'data_horse_horse_type':'horse_type', 'data_horse_img_url':'img_url',
                           'data_horse_last_breeding_reset':'last_breeding_reset', 'data_horse_name':'horse_name',
                           'data_horse_nft_id':'nft_id', 'data_horse_offsprings':'offsprings',
                           'data_horse_race_statistic_first_place_finishes':'first_place_finishes',
                           'data_horse_race_statistic_number_of_races':'number_races',
       'data_horse_race_statistic_second_place_finishes':'second_place_finishes',
       'data_horse_race_statistic_third_place_finishes':'third_place_finishes',
       'data_horse_race_statistic_win_rate':'win_rate'})
horse_data = horse_data.drop(columns=['Unnamed: 0', 'data_horse_inserted_at', 'offsprings'])
horse_data.head()

,bloodline,breed_type,breeding_counter,horse_color,data_horse_gen,horse_type,img_url,last_breeding_reset,horse_name,nft_id,first_place_finishes,number_races,second_place_finishes,third_place_finishes,win_rate
0,Buterin,exclusive,3,Dark Byzantium,Z10,Stallion,https://img.zed.run/horses/5D3954.png,2022-12-23T20:28:07Z,Scrunter S Thompson,154936,11,101,7,10,10.89


In [112]:
# code to merge horse data with sales data
test_merge = pd.merge(horse_data,test_sales, right_on='horse_id', left_on='nft_id', how='left')
test_merge

,bloodline,breed_type,breeding_counter,horse_color,data_horse_gen,horse_type,img_url,last_breeding_reset,horse_name,nft_id,first_place_finishes,number_races,second_place_finishes,third_place_finishes,win_rate,horse_id,time_sold,converted_price
0,Buterin,exclusive,3,Dark Byzantium,Z10,Stallion,https://img.zed.run/horses/5D3954.png,2022-12-23T20:28:07Z,Scrunter S Thompson,154936,11,101,7,10,10.89,154936,464926,15.3876
1,Buterin,exclusive,3,Dark Byzantium,Z10,Stallion,https://img.zed.run/horses/5D3954.png,2022-12-23T20:28:07Z,Scrunter S Thompson,154936,11,101,7,10,10.89,154936,463789,11.6966
